![Linkedyn Logo](https://legal.linkedin.com/apps/settings/wcm/designs/linkedin/katy/global/clientlibs/resources/img/default-share.png) 

<h1>Como acessar e utilizar a API do Linkedyn usando Python</h1>

[Powerpoint](Apresentacao%20Linkedin%20API%20-%20udpated.pptx)

<h3>Grupo de trabalho:</h3>

Ana Leandro - [Linkedyn](https://www.linkedin.com/in/apmlpet/) 

Douglas Sousa - [Linkedyn](https://www.linkedin.com/in/douglas-oliveira-sousa/) 

Fabio Rocha - [Linkedyn](https://www.linkedin.com/in/farocha/) 


<h3>Orientador:</h3>

Anderson Cordeiro - [Linkedyn](https://www.linkedin.com/in/andercordeiro/)

In [ ]:
import json
import random
import requests
import string

# Passo 1: Criar um dicionario de acordo com os argumentos necessarios para usar a API de autorizacao: https://learn.microsoft.com/en-us/linkedin/shared/authentication/authorization-code-flow?tabs=HTTPS1
dict = {"client_id": "enter your id", "client_secret": "enter your client secret", "redirect_uri": "http://localhost:8080" }


# Passo 2: Passar o dicionario para um aqruivo json
with open('credentials.json',"w") as a:
        json.dump(dict,a,indent=4)

#Funcao usada para ler credenciais --- sao retiradas do arquivo json com as credenciais
def ler_credenciais(filename):
    with open(filename) as f:
        credentials = json.load(f)
    return credentials
 
# Chamar a funcao para ler credenciais para retirar as informacoes requeridas 
creds = ler_credenciais('credentials.json')
client_id, client_secret = creds['client_id'], creds['client_secret']
redirect_uri = creds['redirect_uri']

# Funcao para criar um CSRF Token: 
#A CSRF token is a unique, secret, and unpredictable value that is generated by the server-side 
# application and shared with the client. When issuing a 
# request to perform a sensitive action, such as submitting a form, 
# the client must include the correct CSRF token.

def criar_CSRF_token():
    #In Python3, ascii_lowercase is a pre-initialized string used as string constant. 
    #In Python, string ascii_lowercase will give the lowercase letters ‘abcdefghijklmnopqrstuvwxyz’.
    #https://www.geeksforgeeks.org/python-string-ascii_lowercase/
    letras = string.ascii_lowercase 
    #The choice() method returns a randomly selected element from the specified sequence.
    #The sequence can be a string, a range, a list, a tuple or any other kind of sequence.
    #https://www.w3schools.com/python/ref_random_choice.asp
    #The join() method takes all items in an iterable and joins them into one string.
    #https://www.w3schools.com/python/ref_string_join.asp
    #A string must be specified as the separator.
    token = ''.join(random.choice(letras) for i in range(20))
    return token

#Funcao usada para abrir a pagina do linkedin onde faremos o request de autorizacao.
def abrir_url(url):
    #The webbrowser module provides a high-level interface to allow displaying web-based documents to users. 
    #Under most circumstances, simply calling the open() function from this module will do the right thing.
    #https://docs.python.org/3/library/webbrowser.html
    import webbrowser
    print(url)
    webbrowser.open(url)

#Link para acesso a authorizacao de dois fatores 
#https://learn.microsoft.com/en-us/linkedin/shared/authentication/authorization-code-flow?tabs=HTTPS1
api_url = 'https://www.linkedin.com/oauth/v2'
 
#Funcao usada para fazer o HTTP request para o url de autorizacao
#Vamos coletar todos os parametros requeridos e fazer o pedido

def authorizacao(api_url,client_id,client_secret,redirect_uri):
    #chamamos a funcao que cria o token de seguranca CSRF
    csrf_token = criar_CSRF_token()
    #criamos um dicionario com os parametros requeridos de acordo com a documentacao
    #https://learn.microsoft.com/en-us/linkedin/shared/authentication/authorization-code-flow?tabs=HTTPS1
    params = {
        'response_type': 'code',#The OAuth 2.0 authorization code.
        'client_id': client_id,
        'redirect_uri': redirect_uri,
        'state': csrf_token,#A value used to test for possible CSRF attacks.
        'scope': 'r_liteprofile,r_emailaddress,w_member_social' #scope sao os acessos que temos a API --- temos acesso basico
        }
    # fazemos o request ao URL da API de autorizacao e passamos os parametros requeridos
    resposta = requests.get(f'{api_url}/authorization',params=params)
    # Esse request vai me retonar uma pagina para inserir as autentificacoes do linkedin e 
    # checar as autorizacoes que voce tem acesso, assim como checar se voce autoriza o aplicativo a usar os API.
    # Essa pagina sera aberta usando a funcao abrir_url que criamos mais acima.
    abrir_url(resposta.url)
    #Depois que essa pagina foi aberta, quando a gente passa da parte de inserir os seus logins do Linkedin e 
    # de fazer a permissao do aplicativo essa pagina sera redirecionada para a pagina que voce colocou no aplicativo 
    # em "Authorized redirect URLs for your app"
    #Quando essa segunda pagina abrir aparecera um erro -- Esse erro somente acontece porque voce nao tem autorizacao ainda
    #Mas voce vai pegar esse link dessa pagina porque sera usado para retirar o "code" para a segunda etapa da 
    #autentificao de dois fatores 
    #Vamos inserir o link no input abaixo 
    redirect_resposta = input()
    # Nesta etapa vamos retirar apenas o code, que sera o nosso access token.
    # "The access token is found right after the ?code= parameter and before the &state parameter."
    #https://www.jcchouinard.com/authenticate-to-linkedin-api-using-oauth2/
    # Para isso vamos chamar uma funcao
    auth_code = parse_redirect_uri(redirect_resposta)
    return auth_code

# Funcao para coletar o access token ou "code" do link que fomos redirecionados.
def parse_redirect_uri(redirect_response):
    #The URL parsing functions focus on splitting a URL string into its components, or on combining URL components into a 
    # URL string.
    #Parse a query string given as a string argument (data of type application/x-www-form-urlencoded). Data are returned 
    # as a dictionary. 
    # The dictionary keys are the unique query variable names and the values are lists of values for each name.
    #https://docs.python.org/3/library/urllib.parse.html
    from urllib.parse import urlparse, parse_qs
 
    url = urlparse(redirect_response)
    url = parse_qs(url.query) # o code esta presente na partev "query" do link e por isso estamos selecionando essa parte
    print(url) # Aqui temos um dictionaary de listas. 
    return url['code'][0] # Esse access token sera retornado na funcao autorizacao como 'auth_code'

#Funcao para criar headers -- requerimento para fazer um request para a api de autentificacao
#https://www.jcchouinard.com/authenticate-to-linkedin-api-using-oauth2/
#https://www.geeksforgeeks.org/http-headers/

def headers(access_token):
    headers = {
    'Authorization': f'Bearer {access_token}',
    'cache-control': 'no-cache',
    'X-Restli-Protocol-Version': '2.0.0'
    }
    return headers

#Funcao para checar se voce ja tem token e salva-lo, ou apenas retorna-lo se voce ja tiver gerado e salvo o token

def auth(credentials):
    #Re-le para usar na funcao as credenciais
    creds = ler_credenciais(credentials)
    print(creds)
    client_id, client_secret = creds['client_id'], creds['client_secret']
    redirect_uri = creds['redirect_uri']
    api_url = 'https://www.linkedin.com/oauth/v2'
    
    # checa se voce ja tem o access token no seu arquivo json com credenciais
         
    if 'access_token' not in creds.keys(): 
        #Se o access token nao estiver no arquivo de credenciais, essa etapa ira rodar o programa e chamar
        #a funcao que pede o request ao api
        auth_code = authorizacao(api_url,client_id,client_secret,redirect_uri)
        #esse code ou access token tem tempo de duracao
        #por isso vamos usar ele como argumento para a funcao refresh_token que ira trocar esse code pro um novo acess token
        access_token = refresh_token(auth_code,client_id,client_secret,redirect_uri)
        #nesta etapa fazemos update no dicionario que pegamos ao "load" o json de credenciais e fazemos um update
        #com o novo access token
        creds.update({'access_token':access_token})
        #depois de udpated, vamos usar a funcao salvar_token para dump esse novo dicionario dentro do nosso json
        salvar_token(credentials,creds)
    else: 
        access_token = creds['access_token']
    return access_token

#Funcao para refresh o token
#Refresh tokens are used to get a new access token when your current access token expires.
#The main purpose of using a refresh token is to considerably shorten the life of an access token. 
# The refresh token can then later be used to authenticate the user as
# and when required by the application without running into problems such as cookies being blocked, etc.
#https://learn.microsoft.com/en-us/linkedin/shared/authentication/programmatic-refresh-tokens
#https://www.rfc-editor.org/rfc/rfc6749#section-1.5
#https://learn.microsoft.com/en-us/linkedin/shared/authentication/postman-getting-started
def refresh_token(auth_code,client_id,client_secret,redirect_uri):
    access_token_url = 'https://www.linkedin.com/oauth/v2/accessToken'
 
    data = {
        'grant_type': 'authorization_code',
        'code': auth_code, # code coletado na funcao auth
        'redirect_uri': redirect_uri,
        'client_id': client_id,
        'client_secret': client_secret
        }
 
    response = requests.post(access_token_url, data=data, timeout=30)
    response = response.json()
    print(response)
    access_token = response['access_token']
    return access_token

#Funcao para salvar o access token que foi gerado junto as outras credenciais
def salvar_token(filename,data):
    data = json.dumps(data, indent = 4) 
    with open(filename, 'w') as f: 
        f.write(data)


# Essa parte fala para o enviroment do python que  script a ser rodado esta aqui neste aqruivo
#Whenever the Python interpreter reads a source file, it does two things:
#it sets a few special variables like __name__, and then
#it executes all of the code found in the file.
#If your module is the main program, then it will see that __name__ was indeed set to "__main__"
if __name__ == '__main__':
    credentials = 'credentials.json'
    access_token = auth(credentials)
 
